In [1]:
import pandas as pd

# 유지고객 데이터 불러오기
purchase_유지 = pd.read_csv('final_유지_cust.csv')
purchase_유지.head()

,Unnamed: 0,CUSTNO,ASSO,RNO,BBCL,BCL,MCL,SCL,TYPE,PURYEAR,HALF_YEAR,QUARTER,PRICE_TYPE,PURAMT,Q_year
0,0,17485,B,8919510,식품/생활,식품,샐러드채소,양상추,일용품,2015,4,3,저가,2000,2015_3
1,1,17485,B,8919510,식품/생활,식품,김치채소,무우,일용품,2015,4,3,저가,1250,2015_3
2,2,17485,B,8919510,식품/생활,식품,일반한우,한우양지,일용품,2015,4,3,고가,23790,2015_3
3,3,17485,B,8919510,식품/생활,식품,포도,캠벨포도,일용품,2015,4,3,중가,25600,2015_3
4,4,17485,B,8919510,식품/생활,식품,메론,머스크메론,일용품,2015,4,3,중가,12800,2015_3


## 유지고객 데이터 고객별/분기별/카테고리별/금액타입별로 나누기

In [2]:
def pivot(df, aggfunc):
    
    # 분기
    pivot_df_q_ = pd.pivot_table(df,
                               values = 'PURAMT',
                               index = 'CUSTNO',
                               columns = ['Q_year', 'BBCL', 'PRICE_TYPE'],
                               aggfunc = aggfunc)
    pivot_df_q_.fillna(0, inplace = True)
    # 컬럼 변경
    columns_name = []
    for indexs in pivot_df_q_.columns:
         columns_name.append('_'.join([aggfunc, '분기'] + [str(idx) for idx in indexs]))
    pivot_df_q_.columns = columns_name
    
    # 분기 총액
    pivot_df_label_q = pd.pivot_table(df,
                                values = 'PURAMT',
                                index = 'CUSTNO',
                                columns = ['Q_year'],
                                aggfunc = 'sum')
    pivot_df_label_q.fillna(0, inplace = True)

    return pivot_df_q_

In [3]:
df_sum = pivot(purchase_유지, 'sum')
df_sum

,sum_분기_2014_1_가구인테리어/잡화_고가,sum_분기_2014_1_가구인테리어/잡화_저가,sum_분기_2014_1_가구인테리어/잡화_중가,sum_분기_2014_1_가전/레저취미/멀티샵_고가,sum_분기_2014_1_가전/레저취미/멀티샵_저가,sum_분기_2014_1_가전/레저취미/멀티샵_중가,sum_분기_2014_1_교육문화/아동_고가,sum_분기_2014_1_교육문화/아동_저가,sum_분기_2014_1_교육문화/아동_중가,sum_분기_2014_1_식품/생활_고가,...,sum_분기_2015_4_교육문화/아동_중가,sum_분기_2015_4_식품/생활_고가,sum_분기_2015_4_식품/생활_저가,sum_분기_2015_4_식품/생활_중가,sum_분기_2015_4_의류/이미용_고가,sum_분기_2015_4_의류/이미용_저가,sum_분기_2015_4_의류/이미용_중가,sum_분기_2015_4_침구/주방_고가,sum_분기_2015_4_침구/주방_저가,sum_분기_2015_4_침구/주방_중가
CUSTNO,,,,,,,,,,,,,,,,,,,,,
1,0.0,390800.0,0.0,641250.0,0.0,65550.0,0.0,0.0,103740.0,1027500.0,...,0.0,237618.0,5540.0,346870.0,133100.0,0.0,3415160.0,137750.0,0.0,699200.0
2,0.0,562170.0,150000.0,2194330.0,30500.0,0.0,0.0,0.0,113920.0,519290.0,...,0.0,264720.0,0.0,1481370.0,5896250.0,0.0,3002000.0,0.0,0.0,432490.0
3,0.0,600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21800.0,...,0.0,0.0,146169.0,102132.0,0.0,7800.0,0.0,0.0,0.0,0.0
4,0.0,191810.0,189520.0,0.0,0.0,141550.0,0.0,0.0,0.0,8000.0,...,0.0,113357.0,44688.0,617760.0,0.0,90000.0,565980.0,0.0,0.0,54000.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,265680.0,...,0.0,329241.0,134399.0,705709.0,975000.0,157140.0,645900.0,0.0,3800.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,0.0,0.0,0.0,0.0,22000.0,0.0,0.0,3000.0,22700.0,34440.0,...,0.0,12300.0,28650.0,479480.0,0.0,0.0,0.0,0.0,0.0,0.0
19334,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7000.0,11250.0,0.0,...,119200.0,7990.0,52550.0,66170.0,0.0,0.0,0.0,0.0,0.0,0.0
19335,0.0,25000.0,0.0,0.0,0.0,0.0,82800.0,28400.0,103324.0,72130.0,...,684480.0,38480.0,99865.0,829819.0,0.0,188500.0,0.0,0.0,6000.0,0.0


## 분기별 데이터 추출 후 가중치 적용 (2014년 1~4분기 데이터) 

In [7]:
# 가중치 최종 데이터 불러오기
cate_type_weight_final = pd.read_csv('final_weight_data.csv', index_col = 0)
cate_type_weight_final

,가구인테리어/잡화_고가,가구인테리어/잡화_저가,가구인테리어/잡화_중가,가전/레저취미/멀티샵_고가,가전/레저취미/멀티샵_저가,가전/레저취미/멀티샵_중가,교육문화/아동_고가,교육문화/아동_저가,교육문화/아동_중가,식품/생활_고가,식품/생활_저가,식품/생활_중가,의류/이미용_고가,의류/이미용_저가,의류/이미용_중가,침구/주방_고가,침구/주방_저가,침구/주방_중가
1,1.133226,1.038187,1.103678,1.182625,1.062619,1.056405,1.164571,0.936817,1.042577,0.926087,1.034498,0.997115,1.005034,1.063336,1.028013,1.174063,0.968110,1.050982
2,1.010410,1.099600,0.916977,0.980597,0.971917,0.966495,1.141896,1.048001,0.988197,1.166449,0.996508,1.003159,1.061548,1.003590,1.045863,0.919245,0.997292,0.998419
3,1.128867,1.176856,1.082361,1.133007,1.014355,1.274851,1.181695,1.051213,1.168191,0.874482,0.974535,0.947268,1.181886,1.051454,1.295001,1.093370,0.908363,1.076698
4,0.805139,0.782693,0.926362,0.798703,0.957683,0.810162,0.704555,0.973638,0.852609,1.087725,0.996300,1.058663,0.821772,0.899218,0.769882,0.872750,1.158127,0.894326


In [8]:
# 고객별, 타입별 데이터와 분기 입력 시 가중치 적용한 데이터 출력하는 함수 생성
def return_q_df(df, quater):
    if quater == 1:
        df_ = df.iloc[:,:18]
    if quater == 2:
        df_ = df.iloc[:, 18:36]
    elif quater == 3:
        df_ =  df.iloc[:, 36:54]
    elif quater == 4:
        df_ =  df.iloc[:, 54:72]
    
    df_.columns = cate_type_weight_final.columns
    return df_
    
    
def weight_plus(df, quater):
    df_ = return_q_df(df, quater)
    weight_df = df_ * cate_type_weight_final.loc[quater]
    return weight_df

In [9]:
# 함수 적용

# 1분기
weight_plus_q1 = weight_plus(df_sum, 1)

# 2분기
weight_plus_q2 = weight_plus(df_sum, 2)

# 3분기
weight_plus_q3 = weight_plus(df_sum, 3)

# 4분기
weight_plus_q4 = weight_plus(df_sum, 4)

## 유지고객 분기별 카테고리 증감율 적용
- 평균 증감율 기준 -> 감소고객, 증가고객 분류

    - 감소고객 대상 -> *감소율 보다 낮은 고객 : 1(완전 감소고객), 감소율 보다 높은 고객 : 2(보통 감소소객)
    - 증가고객 대상 -> *증가율 보다 높은 고객 : 4(완전 증가고객), 증가율 보다 낮은 고객 : 3(보통 증가고객)
    - 변화 없는 고객 : 0

    *증가율 : 증가금액 평균, 감소율 : 감소금액 평균 

In [37]:
# 데이터 리스트
df_li = [weight_plus_q1 , weight_plus_q2, weight_plus_q3, weight_plus_q4]
total_columns = weight_plus_q1.columns.tolist()

plus_or_minus_cust = []
for i in total_columns:
    feature = i + '_증감여부'
    plus_or_minus_cust.append(feature)
plus_or_minus_cust

['가구인테리어/잡화_고가_증감여부',
 '가구인테리어/잡화_저가_증감여부',
 '가구인테리어/잡화_중가_증감여부',
 '가전/레저취미/멀티샵_고가_증감여부',
 '가전/레저취미/멀티샵_저가_증감여부',
 '가전/레저취미/멀티샵_중가_증감여부',
 '교육문화/아동_고가_증감여부',
 '교육문화/아동_저가_증감여부',
 '교육문화/아동_중가_증감여부',
 '식품/생활_고가_증감여부',
 '식품/생활_저가_증감여부',
 '식품/생활_중가_증감여부',
 '의류/이미용_고가_증감여부',
 '의류/이미용_저가_증감여부',
 '의류/이미용_중가_증감여부',
 '침구/주방_고가_증감여부',
 '침구/주방_저가_증감여부',
 '침구/주방_중가_증감여부']

In [80]:
x= 1
y =2
df_q1 = df_li[x-1]
df_q2 = df_li[y-1]

i =  total_columns[0]
mean_ratio = (df_q2.loc[:, i].sum() - df_q1.loc[:, i].sum())/df_q1.loc[:, i].sum()
mean_ratio

-0.02551046476930298

In [81]:
# 기준 금액 : 기준 원본데이터 * (1+mean_ratio)
stand_price = df_q1.loc[:, i] * (1+mean_ratio)
# 기준금액 컬럼명
stand_column = i + '_기준금액'
# 기준 금액 비교할 데이터프레임에 추가
df_q2[stand_column] = stand_price

# 비교할 원본 데이터 금액 과 기준금액 비교 후 크면 2, 작으면 0, 같으면 1 대입 
# 비교할 원본 금액과 기준 금액 컬럼만 따로 추출
df_whos_ = df_q2[[i, stand_column]]

In [82]:
df_whos_

,가구인테리어/잡화_고가,가구인테리어/잡화_고가_기준금액
CUSTNO,,
1,0.000000,0.0
2,0.000000,0.0
3,0.000000,0.0
4,0.000000,0.0
6,39405.980611,0.0
...,...,...
19333,0.000000,0.0
19334,0.000000,0.0
19335,0.000000,0.0


In [83]:
# 비교할 원본 데이터 금액 과 기준금액 비교 후 크면 2, 작으면 0, 같으면 1 대입 
# 비교할 원본 금액과 기준 금액 컬럼만 따로 추출
df_whos_ = df_q2[[i, stand_column]]

# 증가 고객
df_whos_plus = df_whos_[df_whos_[i] > df_whos_[stand_column]] # 기준금액 보다 큰 경우
# 감소 고객
df_whos_minus = df_whos_[df_whos_[i] < df_whos_[stand_column]] # 기준금액 보다 작은 경우
# 유지 고객
df_whos_same = df_whos_[df_whos_[i] == df_whos_[stand_column]] # 기준금액이랑 차이 없음

In [86]:
# 증증가 고객(증가 고객 데이터)
# 증가 고객의 1분기 총 매출
plus_1q = df_q1.loc[df_whos_plus.index, i].sum()
# 증가 고객의 2분기 총 매출
plus_2q = df_q2.loc[df_whos_plus.index, i].sum()

# 증가고객의 평균 증가율
mean_double_plus_ratio = (plus_2q-plus_1q)/plus_1q
mean_double_plus_ratio

4.216505016862112

In [88]:
# 증가고객을 증증가고객, 보통증가 고객으로 나누기
new_plus_name = '증가고객의 기준_' + i

# 증가고객 데이터프레임에 추가 
# 1분기 증가고객 데이터 * 증감율
df_whos_plus[new_plus_name] =  df_q1.loc[df_whos_plus.index, i] * (1+mean_double_plus_ratio)
df_whos_plus

C:\Users\admin\AppData\Local\Temp\ipykernel_6856\2161200135.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_whos_plus[new_plus_name] =  df_q1.loc[df_whos_plus.index, i] * (1+mean_double_plus_ratio)


,가구인테리어/잡화_고가,가구인테리어/잡화_고가_기준금액,증가고객의 기준_가구인테리어/잡화_고가
CUSTNO,,,
6,3.940598e+04,0.000000,0.000000e+00
10,3.607163e+05,0.000000,0.000000e+00
11,5.506693e+06,807807.646320,4.324246e+06
17,6.313747e+06,0.000000,0.000000e+00
18,4.433173e+05,0.000000,0.000000e+00
...,...,...,...
19300,2.048404e+06,0.000000,0.000000e+00
19304,7.355783e+05,0.000000,0.000000e+00
19306,3.940598e+04,0.000000,0.000000e+00


In [89]:
# 증증가 고객
double_plus = df_whos_plus[df_whos_plus[i]>df_whos_plus[new_plus_name]]
# 보통증가 고객
stand_plus = df_whos_plus[df_whos_plus[i]<=df_whos_plus[new_plus_name]]

double_plus

,가구인테리어/잡화_고가,가구인테리어/잡화_고가_기준금액,증가고객의 기준_가구인테리어/잡화_고가
CUSTNO,,,
6,3.940598e+04,0.000000,0.000000e+00
10,3.607163e+05,0.000000,0.000000e+00
11,5.506693e+06,807807.646320,4.324246e+06
17,6.313747e+06,0.000000,0.000000e+00
18,4.433173e+05,0.000000,0.000000e+00
...,...,...,...
19300,2.048404e+06,0.000000,0.000000e+00
19304,7.355783e+05,0.000000,0.000000e+00
19306,3.940598e+04,0.000000,0.000000e+00


In [90]:
stand_plus

,가구인테리어/잡화_고가,가구인테리어/잡화_고가_기준금액,증가고객의 기준_가구인테리어/잡화_고가
CUSTNO,,,
22,2.555033e+07,8.459894e+06,4.528636e+07
25,1.428517e+05,5.411152e+04,2.896624e+05
26,2.933422e+06,1.406347e+06,7.528267e+06
49,3.059521e+06,1.316898e+06,7.049438e+06
58,3.870274e+05,2.808277e+05,1.503289e+06
...,...,...,...
18896,1.729822e+06,1.325180e+06,7.093774e+06
18964,1.721263e+06,8.747182e+05,4.682423e+06
19003,6.664158e+05,2.827051e+05,1.513338e+06


In [94]:
## 증증가 고객 (4대입)
total_df_ = total_df.copy()
input_column_name = i + '_증감여부'
total_df_.loc[double_plus.index, input_column_name] = 4
total_df_.head(10)

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,식품/생활_저가_증감여부,식품/생활_중가_증감여부,의류/이미용_고가_증감여부,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부
CUSTNO,,,,,,,,,,,,,,,,,,
1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
# total_df에 값 대입
input_column_name = i + '_증감여부'
# 증가면 2
total_df.loc[df_whos_plus.index, input_column_name] = 2
# 감소면 0
total_df.loc[df_whos_minus.index, input_column_name] = 0
# 같으면 1
total_df.loc[df_whos_same.index, input_column_name] = 1

In [69]:
total_df[input_column_name].value_counts()

2    9036
0    8371
1    1459
Name: 가구인테리어/잡화_저가_증감여부, dtype: int64

In [70]:
total_df

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,식품/생활_저가_증감여부,식품/생활_중가_증감여부,의류/이미용_고가_증감여부,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부
CUSTNO,,,,,,,,,,,,,,,,,,
1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19334,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19335,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
df_q2.loc[:, i].sum()

1411471561.2795496

In [75]:
df_q1.loc[:, i].sum()

1376457324.7730763

In [49]:
df_whos_plus.index

Int64Index([    6,    10,    11,    17,    18,    22,    23,    24,    25,
               26,
            ...
            19253, 19260, 19270, 19271, 19296, 19300, 19304, 19306, 19310,
            19312],
           dtype='int64', name='CUSTNO', length=2897)

In [51]:
# 증가면 2
total_df.loc[df_whos_plus.index, input_column_name] = 2

In [52]:
total_df.loc[df_whos_plus.index, input_column_name]

CUSTNO
6        2
10       2
11       2
17       2
18       2
        ..
19300    2
19304    2
19306    2
19310    2
19312    2
Name: 가구인테리어/잡화_고가_증감여부, Length: 2897, dtype: object

In [76]:
# 분기 대입 시 평균 증감율, 감소or 증가 or 유지고객 여부 배출값 생성
def quater_weight_df(x, y): #1분기 대비 2분기 or 1분기 대비 3분기 각 분기 대입
    # 기준 원본 데이터
    df_q1 = df_li[x-1]
    # 비교할 원본 데이터
    df_q2 = df_li[y-1]
    
    # 대입할 데이터프레임 생성
    total_df = pd.DataFrame(columns = plus_or_minus_cust, index = weight_plus_q2.index)
    
    total_ratio = []
    # 각 컬럼명 
    for i in total_columns:
        # 각 컬럼별 평균 증감율 = (비교할 데이터 - 기준 원본 데이터)/기준 원본 데이터
        mean_ratio = (df_q2.loc[:, i].sum() - df_q1.loc[:, i].sum())/df_q1.loc[:, i].sum()

        total_ratio.append(mean_ratio)
        
        # 기준 금액 : 기준 원본데이터 * (1+mean_ratio)
        stand_price = df_q1.loc[:, i] * (1+mean_ratio)
        # 기준금액 컬럼명
        stand_column = i + '_기준금액'
        # 기준 금액 비교할 데이터프레임에 추가
        df_q2[stand_column] = stand_price
        
        # 비교할 원본 데이터 금액 과 기준금액 비교 후 크면 2, 작으면 0, 같으면 1 대입 
        # 비교할 원본 금액과 기준 금액 컬럼만 따로 추출
        df_whos_ = df_q2[[i, stand_column]]
        
        # 증가 고객
        df_whos_plus = df_whos_[df_whos_[i] > df_whos_[stand_column]] # 기준금액 보다 큰 경우
        # 감소 고객
        df_whos_minus = df_whos_[df_whos_[i] < df_whos_[stand_column]] # 기준금액 보다 작은 경우
        # 유지 고객
        df_whos_same = df_whos_[df_whos_[i] == df_whos_[stand_column]] # 기준금액이랑 차이 없음
        
        # total_df에 값 대입
        input_column_name = i + '_증감여부'
        # 증가면 2
        total_df.loc[df_whos_plus.index, input_column_name] = 2
        # 감소면 0
        total_df.loc[df_whos_minus.index, input_column_name] = 0
        # 같으면 1
        total_df.loc[df_whos_same.index, input_column_name] = 1
        
    return total_df, total_ratio

In [77]:
# 함수 대입 (1분기, 2분기)
result_df, total_ratio = quater_weight_df(1, 2)
result_df

,가구인테리어/잡화_고가_증감여부,가구인테리어/잡화_저가_증감여부,가구인테리어/잡화_중가_증감여부,가전/레저취미/멀티샵_고가_증감여부,가전/레저취미/멀티샵_저가_증감여부,가전/레저취미/멀티샵_중가_증감여부,교육문화/아동_고가_증감여부,교육문화/아동_저가_증감여부,교육문화/아동_중가_증감여부,식품/생활_고가_증감여부,식품/생활_저가_증감여부,식품/생활_중가_증감여부,의류/이미용_고가_증감여부,의류/이미용_저가_증감여부,의류/이미용_중가_증감여부,침구/주방_고가_증감여부,침구/주방_저가_증감여부,침구/주방_중가_증감여부
CUSTNO,,,,,,,,,,,,,,,,,,
1,1,0,2,2,1,0,1,1,0,2,2,0,0,1,2,0,1,0
2,1,0,2,0,0,2,1,1,2,2,1,0,0,1,2,0,1,0
3,1,2,1,1,1,1,1,1,1,2,2,2,1,1,2,1,1,1
4,1,0,2,2,2,0,1,1,1,2,2,2,2,0,2,1,2,0
6,2,2,2,1,2,1,1,1,1,0,0,2,0,2,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19333,1,2,1,1,0,1,1,0,2,2,0,2,1,1,0,1,0,1
19334,1,1,1,1,2,1,1,2,2,2,2,2,1,0,1,1,2,1
19335,1,2,1,1,2,2,2,2,2,2,2,2,1,2,2,1,0,2


In [78]:
total_ratio

[-0.02551046476930298,
 0.025437938304586214,
 0.027290352314949688,
 0.018760837176156113,
 -0.05230878332265571,
 0.05705728272700874,
 -0.14351507964009386,
 0.00014522791626715815,
 0.05035778971731079,
 0.04513214240509725,
 0.010709257510890597,
 0.029707741141324386,
 0.029405497701557155,
 0.061714912423814584,
 -0.008485950139132227,
 0.03887141418928895,
 0.007968604005723334,
 -0.025176185333691165]